In [92]:
from __future__ import division, print_function

import matplotlib.pyplot as plt
import matplotlib.lines as mlines
#import seawater.gibbs as gsw

import netCDF4 as nc
import numpy as np
import scipy.io as sio
import pandas as pd
import pickle as pl
import matplotlib.pylab as pylab
params = {'legend.fontsize': 'x-large',
          'figure.figsize': (15, 5),
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large'}
pylab.rcParams.update(params)

from salishsea_tools import nc_tools
from salishsea_tools import viz_tools
from salishsea_tools import  psu_tools

from matplotlib.pyplot import *
#from seabird.cnv import fCNV

from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.basemap import Basemap

%matplotlib inline

### Read the file.txt with pandas

In [93]:
data = pd.read_csv('/home/mgrenier/Documents/GEOTRACES_ARCTIC/GEOTRACES2015-Legs2b3b_ODV_forPaTh.txt',sep='\t')
#data
#data.head() # Displays the head of the table
#data.tail() # Displays the tail of the table

/home/mgrenier/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (0,10,24,26,27,28,29,31,32,33,34,37,38,40,41,42,43) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Rename and redefine variables

In [94]:
data.rename(columns={'yyyy-mm-ddThh:mm:ss.ss':'date',
                     'Longitude [degrees_east]':'lon','Latitude [degrees_north]':'lat',
                     'Bot. Depth [m]':'z_bottom','PRES_01 [decibars]':'P','Depth [metres]':'d',
                     'TE90_01 [degrees C]':'temp','PSAL_01 [psu]':'sal','SIGT_01 [kg/m**3]':'dens',
                     '231-Pa (fg/kg)':'Pa','230-Th (fg/kg)':'Th'}, inplace= True)

# Define the number of columns and rows to display
pd.options.display.max_columns = 64
pd.options.display.max_columns = 94

#data.head(n=2) # Displays the 2 first rows of the head of the table
#data.tail(n=2) # Displays the 2 first rows of the tail of the table

# Redefine the variables with a handy name
Station = data.Station
date = data.date
lon = data.lon
lat = data.lat
P = data.P
d = data.d
t = data.temp
s = data.sal

print(t.values,s.values,d.values,'\n') # Returns Series as ndarray or ndarray-like depending on the dtype 
                           # Quotes around the numbers indicate that the type is an object, not a float

['6.327' '6.3266' '6.326' ..., -1.177 -1.176 -1.176] ['33.3184' '33.3171' '33.3176' ..., 32.804 32.808 32.811] [   6.94     7.932    8.924 ...,  139.493  140.482  141.471] 



### Convert pandas objects into floats (pd.to_numeric) to use the psu_tools.calculate_density(t,s) function

In [95]:
# As the dtype of t and s is "object", we need to convert them in "float" 
# - using the pandas function pd.to_numeric - to calculate the associated potential density
t = pd.to_numeric(t, errors='coerce')
s = pd.to_numeric(s, errors='coerce')
print(t.shape,s.shape,t.values,s.values,'\n')

(106751,) (106751,) [ 6.327   6.3266  6.326  ..., -1.177  -1.176  -1.176 ] [ 33.3184  33.3171  33.3176 ...,  32.804   32.808   32.811 ] 



### Calculate potential density @ P=0 using the function psu_tools.calculate_density(temp, sal) from the salishsea_tools package

#### Description of the function psu_tools.calculate_density(temp, sal) from the salishsea_tools package

In [96]:
psu_tools.calculate_density?

In [97]:
rho = psu_tools.calculate_density(t,s)
rho = rho - 1000
rhomin=rho.min()
rhomax=rho.max()
print(rhomin,rhomax,rho.shape,rho.values,'\n')

6.72009798136 28.056607565 (106751,) [ 26.18296445  26.18198966  26.18246086 ...,  26.37925099  26.38246402
  26.38489664] 



### Inserts the calculated variable in the Series, that is, in your table

In [98]:
idx = 25 # Number of the column where you want to insert your new column, starting from 0 
col_name = 'rho'
value = rho
data.insert(idx, col_name, value)

#Display the table with the new column 'rho' inserted
data.head(2)

,Cruise,Station,Type,date,lon,lat,z_bottom,P,d,Cast,Cast name,Pressure,Pa,231-Pa error,Th,230-Th error,231-Pa/230-Th,Pa/Th error,eNd,Erreur sur eNd,[Nd]pmol/kg,Erreur sur [Nd],Nd/yb,Ce/Ce*,temp,rho,TRAN_01 [%],FLOR_01 [mg/m**3],sal,dens,SPVA_01 [m**3/kg],VAIS_01 [s^-2],SIGO_01 [kg/m**3],POTM_01 [degrees C],SIGP_01 [kg/m**3],FRET_01 [degrees C],DOXY_01 [ml/l],CDOM_01 [mg/m**3],NTRA_01 [mmol/m**3],PSAR_01 [ueinsteins/s/m**2],SPAR_01 [ueinsteins/s/m**2],ASAL [g/kg],CONT [deg C],D_CT [kg/m^3],D0CT [kg/m^3],Conductivity [uS/cm],"Oxygen, SBE 43 [umol/Kg]","Oxygen, SBE 43 [ml/l]","Oxygen, SBE 43 [% saturation]","Oxygen Saturation, Garcia & Gordon [ml/l]","Oxygen Saturation, Weiss [ml/l]","Oxygen raw, SBE 43 [V]",flag,Unnamed: 52,Unnamed: 53,Nitrite-1 [?mol/L],Nitrite-2 [?mol/L],Nitrate-1 [?mol/L],Nitrate-2 [?mol/L],Phosphate-1 [?mol/L],Phosphate-2 [?mol/L],Silicate-1 [?mol/L],Silicate-2 [?mol/L],Ammonium-1 [?mol/L],Ammonium-2 [?mol/L]
0,Arctic GEOTRACES 2015 Leg 2,K1,C,2014-07-15T14:42:00.00,-53.369283,56.120867,3309.0,7,6.940,NaN,TM4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.327,26.182964,NaN,NaN,33.3184,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33165.0,308.472,7.08798,102.23218,6.93322,6.94424,2.5814,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Arctic GEOTRACES 2015 Leg 2,K1,C,2014-07-15T14:42:00.00,-53.369283,56.120867,3309.0,8,7.932,NaN,TM4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.3266,26.181990,NaN,NaN,33.3171,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33164.0,306.462,7.04178,101.56426,6.93332,6.94435,2.5744,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Redefine the variables with a handy name where Pa or Th data exist

In [99]:
PaData=data[data.Pa.notnull()] # PaData: select only rows where Pa is not NaN -> n=92
ThLost=PaData[PaData.Th.isnull()] # Among PaData, select only rows where Th is NaN 
                                  # (refer to Th sample lost)-> n=2
PaThData=pd.concat([data[data.Th > 0],PaData[PaData.Th.isnull()]]) # Concatenate the rows where Th is not NaN
                                                                   # and where Th is NaN but Pa is not NaN -> n=94
print(len(PaData),len(PaThData),'\n')
#print(PaThData[PaThData.Station == 'BB3']) # Displays the rows of the table corresponding to the Pa and Th data 
                                           # of the station BB3
PaTh_sta = PaThData.Station
PaTh_lon = PaThData.lon
PaTh_lat = PaThData.lat
PaTh_t = PaThData.temp
PaTh_s = PaThData.sal
PaTh_rho = PaThData.rho
PaTh_d = PaThData.d


92 94 



### Do a list of all the stations and another one of the Pa Th stations only

In [100]:
listAllSta = []
listPaThSta = []

for i in range(0,len(lon-1),1):
    if station[i] not in listAllSta:
        listAllSta.append(station[i])
    if data.Pa.notnull()[i]:
        if station[i] not in listPaThSta:
            listPaThSta.append(station[i])

In [101]:
listPaThSta,listPaThSta[1]
#listAllSta

(['K1', 'LS2', 'BB1', 'BB3', 'BB2', 'CB2', 'CB3', 'CB4', '308/CAA8'], 'LS2')

### Example of a dictionnary built through a loop, with another writing structure

In [102]:
{stn: listPaThSta.index(stn) for stn in listPaThSta}

{'308/CAA8': 8,
 'BB1': 2,
 'BB2': 4,
 'BB3': 3,
 'CB2': 5,
 'CB3': 6,
 'CB4': 7,
 'K1': 0,
 'LS2': 1}

### Sort the data of each station not following cast but following increasing depth

In [107]:
from collections import OrderedDict

d_sort = OrderedDict()
for stn in listPaThSta:
    d_sort[stn] = PaThData[PaThData.Station == stn].sort_values(by='d', ascending=[True])
PaThDataSorted = pd.concat(d_sort.values())
d_sort[listPaThSta[0]] ## This line and the line below are equivalent
d_sort['K1']
#PaThDataSorted

,Cruise,Station,Type,date,lon,lat,z_bottom,P,d,Cast,Cast name,Pressure,Pa,231-Pa error,Th,230-Th error,231-Pa/230-Th,Pa/Th error,eNd,Erreur sur eNd,[Nd]pmol/kg,Erreur sur [Nd],Nd/yb,Ce/Ce*,temp,rho,TRAN_01 [%],FLOR_01 [mg/m**3],sal,dens,SPVA_01 [m**3/kg],VAIS_01 [s^-2],SIGO_01 [kg/m**3],POTM_01 [degrees C],SIGP_01 [kg/m**3],FRET_01 [degrees C],DOXY_01 [ml/l],CDOM_01 [mg/m**3],NTRA_01 [mmol/m**3],PSAR_01 [ueinsteins/s/m**2],SPAR_01 [ueinsteins/s/m**2],ASAL [g/kg],CONT [deg C],D_CT [kg/m^3],D0CT [kg/m^3],Conductivity [uS/cm],"Oxygen, SBE 43 [umol/Kg]","Oxygen, SBE 43 [ml/l]","Oxygen, SBE 43 [% saturation]","Oxygen Saturation, Garcia & Gordon [ml/l]","Oxygen Saturation, Weiss [ml/l]","Oxygen raw, SBE 43 [V]",flag,Unnamed: 52,Unnamed: 53,Nitrite-1 [?mol/L],Nitrite-2 [?mol/L],Nitrate-1 [?mol/L],Nitrate-2 [?mol/L],Phosphate-1 [?mol/L],Phosphate-2 [?mol/L],Silicate-1 [?mol/L],Silicate-2 [?mol/L],Ammonium-1 [?mol/L],Ammonium-2 [?mol/L]
6440,2015002,K1,C,2015-07-14T13:13:35.00,-53.370000,56.121600,3309.0,10,9.909,2.0,Geo-AN1,7.0,0.36,0.06,1.16,0.04,0.71,0.06,NaN,NaN,NaN,NaN,NaN,NaN,6.656,26.151050,93.323,0.554,33.332,26.197,185.508,NaN,26.151,6.657,26.151,-1.83,7.199,5.616,NaN,NaN,543.639,33.49,6.67,26.199,26.153,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,7.260,0.07,NaN,0.13,NaN,0.10,NaN,0.00,NaN,0.00,NaN
11151,2015002,K1,C,2015-07-14T20:32:39.00,-53.383000,56.117700,3312.0,50,49.539,4.0,Geo-AN2,50.0,0.30,0.05,1.25,0.06,0.56,0.04,NaN,NaN,NaN,NaN,NaN,NaN,4.543,27.455188,92.123,6.235,34.657,27.687,62.194,0,27.455,4.54,27.455,-1.94,7.672,5.959,NaN,NaN,255.060,34.821,4.537,27.689,27.457,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6581,2015002,K1,C,2015-07-14T13:13:35.00,-53.370000,56.121600,3309.0,151,149.570,2.0,Geo-AN1,151.0,0.59,0.06,3.03,0.08,0.44,0.02,NaN,NaN,NaN,NaN,NaN,NaN,3.454,27.677618,100.143,0.031,34.794,28.382,41.847,0,27.678,3.445,27.679,-2.02,6.764,5.897,NaN,NaN,512.100,34.96,3.442,28.383,27.68,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,151.282,0.30,NaN,14.81,NaN,0.97,NaN,7.54,NaN,0.47,NaN
11405,2015002,K1,C,2015-07-14T20:32:39.00,-53.383000,56.117700,3312.0,304,301.009,4.0,Geo-AN2,304.0,0.91,0.12,3.09,0.08,0.67,0.04,NaN,NaN,NaN,NaN,NaN,NaN,3.377,27.715411,100.561,0.011,34.832,29.131,39.49,0,27.715,3.358,27.717,-2.14,6.829,6.007,NaN,NaN,205.432,34.998,3.354,29.133,27.719,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11606,2015002,K1,C,2015-07-14T20:32:39.00,-53.383000,56.117700,3312.0,505,499.790,4.0,Geo-AN2,505.0,0.31,0.08,3.56,0.09,0.20,0.02,NaN,NaN,NaN,NaN,NaN,NaN,3.393,27.727413,100.657,0.011,34.849,30.074,39.998,0,27.727,3.36,27.731,-2.29,6.798,5.859,NaN,NaN,154.829,35.016,3.356,30.077,27.733,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11808,2015002,K1,C,2015-07-14T20:32:39.00,-53.383000,56.117700,3312.0,707,699.366,4.0,Geo-AN2,707.0,0.79,0.08,3.55,0.10,0.51,0.02,NaN,NaN,NaN,NaN,NaN,NaN,3.375,27.733148,100.677,0.01,34.854,31.012,41.044,0,27.733,3.327,27.738,-2.45,6.822,5.793,NaN,NaN,142.360,35.021,3.323,31.015,27.74,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12113,2015002,K1,C,2015-07-14T20:32:39.00,-53.383000,56.117700,3312.0,1012,1000.344,4.0,Geo-AN2,1012.0,1.48,0.10,3.85,0.10,0.88,0.03,NaN,NaN,NaN,NaN,NaN,NaN,3.366,27.735615,100.711,0.01,34.856,32.414,43.18,0,27.736,3.294,27.743,-2.68,6.838,5.913,NaN,NaN,152.250,35.023,3.291,32.417,27.745,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12621,2015002,K1,C,2015-07-14T20:32:39.00,-53.383000,56.117700,3312.0,1520,1500.681,4.0,Geo-AN2,1520.0,0.15,0.06,3.90,0.12,0.09,0.01,NaN,NaN,NaN,NaN,NaN,NaN,3.45,27.743379,100.774,0.008,34.876,34.729,46.637,0,27.743,3.335,27.755,-3.06,6.712,5.959,NaN,NaN,169.337,35.044,3.331,34.733,27.757,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5078,Arctic GEOTRACES 2015 Leg 2,K1,C,2014-07-15T08:09:00.00,-53.366700,56.120117,3311.0,2026,1999.135,NaN,TM3,2

### Calculate potential density to display isopycnal, by contour

In [115]:
import DerivVar
from importlib import reload

reload(DerivVar)
isop,si,ti = DerivVar.isopycnal_forTSplot(s,t)

isopmin=isop.min()
isopmax=isop.max()
levels = np.arange(rhomin,rhomax,0.2) 
# Plot data ***********************************************
#CS = plt.contour(si,ti,dens, linestyles='dashed', colors='k')
#plt.clabel(CS, fontsize=12, inline=1, fmt='%1.0f') # Label every second level
isopmin,isopmax,isop.shape
#levels

271.0 13.0


/ocean/mgrenier/geotraces/research/Geotraces2015data/DerivVar.py:17: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  isop = np.zeros((ydim,xdim))


(6.3859208842368389, 28.482608102588529, (13, 271))